### Importação de bibliotecas

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from category_encoders import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

# 1. Exploração da Base de Dados

## 1.1. Visualização das Primeiras Linhas

In [ ]:
data_gas = pd.read_csv("../data/month_2.csv")
data_gas.head(11)

## 1.2. Verificação dos Tipos de Cada Coluna

#### 1.2.1 Quantas linhas e colunas tem na tabela:


In [ ]:
data_gas.shape

A tabela possui 14 colunas e 381274 linhas

In [ ]:
data_gas.describe()

Descrição das colunas numéricas e categóricas antes da normalização das variáveis.

#### 1.2.2. Quantos tipos de dados tem na tabela:


In [ ]:
type_counts = data_gas.dtypes.value_counts()
print(type_counts)

A tabela tem 8 colunas de tipo **float** (numeros reais), 5 colunas de tipo **object** (letras e numeros) e 1 coluna do tipo **int** (números inteiros)

#### 1.2.3. Tipos de dados por colunas:

In [ ]:
data_gas.dtypes


Esse bloco de código mostra o nome da coluna e o seu tipo.

#### 1.2.4. Filtro de tipos:

In [ ]:
# substitua por float64, object ou int64
data_gas.select_dtypes(include='float64')

Ao trocar o que esta entre '', pode se ver as primeiras e ultimas linhas de todas as colunas do tipo solicitado.

Os tipos podem ser:
* float64 --> float
* int64 --> int
* object

## 1.3. Análise Gráfica
#### 1.3.1. Acoplamento de todas as tabelas

&nbsp;&nbsp;&nbsp;&nbsp; Para análise minuciosa e de melhor visualização, fez-se necessária a ação de acoplar todas as tabelas, uma vez que as seguintes análises gráficas e estatísticas só foram possíveis através da mesma. Dessa forma, todos os dados dos meses 2, 3, 4 e 6 estão reunidos em um único *DataFrame*.

In [73]:
mes_02 = pd.read_csv('../data/month_2.csv')
mes_03 = pd.read_csv('../data/month_3.csv')
mes_04 = pd.read_csv('../data/month_4.csv')
mes_05 = pd.read_csv('../data/month_5.csv')
mes_06 = pd.read_csv('../data/month_6.csv')
cadastro = pd.read_csv('../data/informacao_cadastral.csv')
meses = pd.concat([mes_02, mes_03, mes_04, mes_05, mes_06])
resultado = pd.merge(meses, cadastro, on=['clientCode', 'clientIndex'], how='inner')
resultado.to_csv('resultado.csv', index=False)


#### 1.3.1.1. Gráfico de Linhas

&nbsp;&nbsp;&nbsp;&nbsp; Visando encontrar padrões, gráficos de linha foram feitos para análise de mínimos e máximos (picos) valores. As colunas utilizadas nessa análise são: _**pulseCount**_, _**initialIndex**_, _**meterIndex**_, pois metrificam o consumo do gás e a pulsação contabilizada pelo medidor.

In [ ]:
resultado.pulseCount.plot(kind='hist', bins=100, color='pink', edgecolor='purple')

In [ ]:
resultado.initialIndex.plot(kind='hist', bins=100, color='pink', edgecolor='purple')

In [ ]:
resultado.meterIndex.plot(kind='hist', bins=100, color='pink', edgecolor='purple')

&nbsp;&nbsp;&nbsp;&nbsp; A partir desses gráficos, foi possível identificar como os padrões atuam e funcionam, além de visibilizar números distoantes daqueles considerados na média para cada coluna, fazendo-nos prestar maior atenção nos mesmos para possíveis anomalias.

#### 1.3.1.2. Gráfico de Blocos

&nbsp;&nbsp;&nbsp;&nbsp; Visando identificar a frequência de cada variável/dado específico do *DataFrame*, foram criados gráficos de bloco/vela para as colunas: _**perfil_consumo**_ e _**clientCode**_.

In [ ]:
perfil_counts = resultado['perfil_consumo'].value_counts()
sns.barplot(x=perfil_counts.index, y=perfil_counts.values)
plt.xlabel('Perfil de Consumo')
plt.ylabel('Contagem')
plt.title('Contagem de Perfis')
plt.xticks(rotation=55)
plt.show()

In [ ]:
data_gas.meterIndex.plot(kind='line')

client_counts = resultado['clientCode'].value_counts()
sns.barplot(x=client_counts.index, y=client_counts.values)
plt.xlabel('Código do Cliente')
plt.ylabel('Contagem')
plt.title('Contagem dos Códigos')
plt.xticks(rotation=90)
plt.gcf().subplots_adjust(bottom=0.3, top=0.9)
plt.xticks(ticks=range(0, len(client_counts), 40), labels=client_counts.index[::40])
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; A partir dos resultados, foi possível ver o nível de frequências dos dados presentes para cada código de cliente, bem como a quantidade de clientes por perfil de consumo específico, dando-nos *insights* sobre os gastos de casa.

#### 1.3.1.3. Gráfico de Dispersão

&nbsp;&nbsp;&nbsp;&nbsp; Planejando encontrar e analisar uma possível correlação entre variáveis, foram criados gráficos de dispersão para identificar a relação entre dados que guiassem a equipe às suas primeiras hipóteses. Para isso, foram utilizadas as colunas: _**cidade**_, _**perfil_consumo**_ e _**bairro**_.

In [ ]:

df = pd.DataFrame(resultado)
sns.scatterplot(x='cidade', y='perfil_consumo', data=df)
plt.xlabel('Cidade')
plt.ylabel('Perfil de Consumo')
plt.title('Gráfico de Dispersão')
plt.xticks(rotation=45)
plt.show()

In [ ]:

df = pd.DataFrame(resultado)
sns.scatterplot(x='bairro', y='perfil_consumo', data=df)
plt.xlabel('Bairro')
plt.ylabel('Perfil de Consumo')
plt.title('Gráfico de Dispersão')
plt.xticks(rotation=80)
plt.show()

In [ ]:

df = pd.DataFrame(resultado)
sns.scatterplot(x='categoria', y='perfil_consumo', data=df)
plt.xlabel('Categoria')
plt.ylabel('Perfil de Consumo')
plt.title('Gráfico de Dispersão')
plt.xticks(rotation=80)
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Com base nos resultados obtidos, foi possível encontrar um padrão no perfil do consumidor com base em sua localidade, o que facilita o treinamento do modelo em futuras pedições. Também foi possível identificar dados incorretos que acabam deturpando a análise como: **Gravatai** e **Gravataí** - representando a mesma cidade - e **Passo da Areia** e **Passo D'areia** - representando o mesmo bairro.

#### 1.3.1.4. Gráfico de Setor

&nbsp;&nbsp;&nbsp;&nbsp; Já para dados mais técnicos e com menores inconsistências de frequência, foram utilizados gráficos de setor para relatar a porcentagem de cada um. Para análise, as colunas _**model**_ e _**inputType**_ foram selecionadas.

In [ ]:
df = pd.DataFrame(resultado)
counts = df['model'].value_counts()
colors = ['#E6E6FA', '#DDA0DD']
plt.pie(counts, labels=counts.index, colors=colors[:len(counts)], autopct='%1.1f%%', startangle=140)
plt.axis('equal') 
plt.title('Distribuição por Porcentagem')
plt.show()

In [ ]:
df = pd.DataFrame(resultado)
counts = df['inputType'].value_counts()
colors = ['#E6E6FA', '#DDA0DD', '#D8BFD8', '#E1BEE7', '#CDA4DE',
          '#FFC0CB', '#FFB6C1', '#FADADD', '#F5DEB3', '#FFDFDF',
          '#FFDAB9', '#F08080', '#F6B5B1', '#F4A460']
plt.pie(counts, labels=counts.index, colors=colors[:len(counts)], autopct='%1.1f%%', startangle=140)
plt.axis('equal') 
plt.title('Distribuição por Porcentagem')
plt.show()

In [ ]:
df = pd.DataFrame(resultado)
counts = df['categoria'].value_counts()
colors = ['#E6E6FA', '#DDA0DD', '#D8BFD8', '#E1BEE7', '#CDA4DE',
          '#FFC0CB', '#FFB6C1', '#FADADD', '#F5DEB3', '#FFDFDF',
          '#FFDAB9', '#F08080', '#F6B5B1', '#F4A460']
plt.pie(counts, labels=counts.index, colors=colors[:len(counts)], autopct='%1.1f%%', startangle=140)
plt.axis('equal') 
plt.title('Distribuição por Porcentagem')
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp; Com os gráficos obtidos, é possível agora a visualização do nível de incidência de cada tipo de leitura, medidor e categorização de cliente, assim, sendo viável a correlação desses fatores posteriormente com outros índices.

# 2. Pré processamento de dados

**Leitura dos dados:** Na célula abaixo, o *DataFrame* referente às informações cadastrais é carregado, assim como os relacionados aos meses 2, 3, 4, 5 e 6. Em seguida, todos os meses são concatenados para serem juntados com as informações cadastrais, formando um único *DataFrame*.


In [85]:
mes_02 = pd.read_csv('../data/month_2.csv')
mes_03 = pd.read_csv('../data/month_3.csv')
mes_04 = pd.read_csv('../data/month_4.csv')
mes_05 = pd.read_csv('../data/month_5.csv')
mes_06 = pd.read_csv('../data/month_6.csv')
cadastro = pd.read_csv('../data/informacao_cadastral.csv')

meses = pd.concat([mes_02, mes_03, mes_04, mes_05, mes_06])

df = pd.merge(meses, cadastro, on=['clientCode', 'clientIndex',], how='left')
df.to_csv('resultado.csv', index=False)

In [ ]:
df = df.drop(columns=['initialIndex', 'meterSN', 'cep', 'cidade', 'contratacao', 'situacao', 'condIndex' ])
df

## 2.1. Identificação de outliers

&nbsp;&nbsp;&nbsp;&nbsp; A identificação dos *outliers* é importantíssima para o desenvolvimento do modelo preditivo. Abaixo temos os códigos que foram utilizados para identificar os dados anormais.

In [ ]:
sns.boxplot(df['meterIndex'])

&nbsp;&nbsp;&nbsp;&nbsp; Após interpretarmos o gráfico gerado com o comando acima, notamos dois pontos muito destoantes dos demais, um no meio do gráfico e outro bem acima de todos os demais. Para identificarmos os códigos desses clientes e realizarmos uma futura interpretação, é preciso dos códigos abaixo:

In [ ]:
sns.boxplot(df[df['meterIndex']>200000]['meterIndex'])

In [89]:
outlier1 = df[df['meterIndex'] > 200000]
outlier1 = outlier1[outlier1['meterIndex'] < 400000]

In [ ]:
outlier1['clientCode'].unique()

&nbsp;&nbsp;&nbsp;&nbsp; Após identificarmos o primeiro *outlier*, conseguimos o "clientCode" desse usuário, que no caso é "1985ad3cff2f49f0a7743ce6aedb725ce078466631cbf3d2d799a5d5c2f0d1ef". Guardarmos esse código é extremamente importante para que possamos identificar esse usuário novamente no futuro. Com o código abaixo podemos gerar um gráfico com o consumo desse usuário, a fim de analisá-lo individualmente.

In [ ]:
dado = df[df['clientCode']=='1985ad3cff2f49f0a7743ce6aedb725ce078466631cbf3d2d799a5d5c2f0d1ef']
plt.plot(dado['datetime'],dado['meterIndex'])
len(dado['clientCode']) 

&nbsp;&nbsp;&nbsp;&nbsp; Agora, com o segundo *outlier*, faremos o mesmo procedimento. Seguindo os mesmos passos, descobrimos que o "clientCode" desse usuário que encontrasse bem acima de todos os outros é "a2075145d3cc47b2b56aeec5e9c78fe7e0055169961b6823629772c96f1f0319". Para o analisarmos de forma individual, precisaremos do mesmo código utilizado com o primeiro *outlier*.

In [ ]:
dado = df[df['clientCode']=='1985ad3cff2f49f0a7743ce6aedb725ce078466631cbf3d2d799a5d5c2f0d1ef']
plt.plot(dado['datetime'],dado['meterIndex'])
len(dado['clientCode']) 

## 2.1.2. Tratamento de *Missing Values*

### 2.1.2.1. Idendificação dos *Missing Values*

&nbsp;&nbsp;&nbsp;&nbsp; Na célula abaixo, é apresentado a quantidade de dados nulos por coluna. Nesse sentido, as colunas *'gatewayGeoLocation.alt'*, *'gatewayGeoLocation.lat'*, *'gatewayGeoLocation.long'* e *'rssi'* exibem uma quantidade notável de valores nulos em suas linhas, enquanto as colunas 'bairro', 'categoria', 'perfil_consumo' e 'condCode' também exibem alguns dados nulos, porém em menor quantidade. Como é evidenciado na *Series* a seguir:

In [ ]:
dados_nulos_coluna = df.isnull().sum()
dados_nulos_coluna

&nbsp;&nbsp;&nbsp;&nbsp; Com a finalidade de uma análise mais aprofundada, criou-se um DataFrame que mostra a porcentagem de dados ausentes em cada mês observado. Nesse sentido, identifica-se que nos meses 2 e 3, os dados das presentes colunas estão 100% ausentes, enquanto, a partir do mês 4, a porcentagem começa a diminuir, embora ainda permaneça significativa. Outro ponto a ser destacado é que, nos meses 5 e 6, a coluna *rssi* não apresenta dados ausentes, como é evidenciado a seguir:

In [ ]:
df_nulos_mes = pd.DataFrame({
    'Colunas_com_dados_nulos': ['rssi','gatewayGeoLocation.alt','gatewayGeoLocation.lat','gatewayGeoLocation.long'],
    'Mes_2 (%)':[ ((mes_02['rssi'].isnull().sum())/len(mes_02))*100, ((mes_02['gatewayGeoLocation.alt'].isnull().sum())/len(mes_02))*100, ((mes_02['gatewayGeoLocation.lat'].isnull().sum())/len(mes_02))*100, ((mes_02['gatewayGeoLocation.long'].isnull().sum())/len(mes_02))*100],
    'Mes_3 (%)':[ ((mes_03['rssi'].isnull().sum())/len(mes_03))*100, ((mes_03['gatewayGeoLocation.alt'].isnull().sum())/len(mes_03))*100, ((mes_03['gatewayGeoLocation.lat'].isnull().sum())/len(mes_03))*100, ((mes_03['gatewayGeoLocation.long'].isnull().sum())/len(mes_03))*100],
    'Mes_4 (%)':[ ((mes_04['rssi'].isnull().sum())/len(mes_04))*100, ((mes_04['gatewayGeoLocation.alt'].isnull().sum())/len(mes_04))*100, ((mes_04['gatewayGeoLocation.lat'].isnull().sum())/len(mes_04))*100, ((mes_04['gatewayGeoLocation.long'].isnull().sum())/len(mes_04))*100],
    'Mes_5 (%)':[ ((mes_05['rssi'].isnull().sum())/len(mes_05))*100, ((mes_05['gatewayGeoLocation.alt'].isnull().sum())/len(mes_05))*100, ((mes_05['gatewayGeoLocation.lat'].isnull().sum())/len(mes_05))*100, ((mes_05['gatewayGeoLocation.long'].isnull().sum())/len(mes_05))*100],
    'Mes_6 (%)':[ ((mes_06['rssi'].isnull().sum())/len(mes_06))*100, ((mes_06['gatewayGeoLocation.alt'].isnull().sum())/len(mes_06))*100, ((mes_06['gatewayGeoLocation.lat'].isnull().sum())/len(mes_06))*100, ((mes_06['gatewayGeoLocation.long'].isnull().sum())/len(mes_06))*100],

})

df_nulos_mes

&nbsp;&nbsp;&nbsp;&nbsp; Além desses pontos, na coluna *rssi* foram identificados alguns valores positivos. No entanto, de acordo com a documentação do banco de dados fornecido, todos os valores deveriam ser negativos.

In [ ]:
valores_positivos = df[df['rssi']>0]
valores_positivos

In [ ]:
total_valores_positivos = len(df[df['rssi']>0])
total_valores_positivos

&nbsp;&nbsp;&nbsp;&nbsp; Além disso, a *Series* abaixo indica a porcentagem de dados nulos das colunas 'bairro', 'categoria', 'perfil_consumo' e 'condCode' em relação a quantidade geral dos dados. Assim, conclui-se que os dados ausentes dessas colunas são inferiores à 5%

In [ ]:
dicionario_relacao = {'bairro': ((df['bairro'].isnull().sum())/len(df))*100, 'categoria': ((df['categoria'].isnull().sum())/len(df))*100, 'perfil_consumo': ((df['perfil_consumo'].isnull().sum())/len(df))*100 , 'condCode': ((df['condCode'].isnull().sum())/len(df))*100}
series_nulos_geral = pd.Series(dicionario_relacao)

series_nulos_geral

### 2.1.2.2. Tratamento dos *Missing Values*

 &nbsp;&nbsp;&nbsp;&nbsp; Dessa maneira, com o objetivo de evitar o enviesamento dos dados e aumentar a eficácia do modelo preditivo, é necessário tratar esses valores. Nesse contexto, com base na análise feita no tópico anterior, as colunas *gatewayGeoLocation.alt*, *gatewayGeoLocation.lat* e *gatewayGeoLocation.long* foram removidas do *DataFrame*, tendo em vista o grande porcentual de dados nulos.

In [98]:
df.drop(columns=['gatewayGeoLocation.alt','gatewayGeoLocation.lat', 'gatewayGeoLocation.long'], inplace=True)

 &nbsp;&nbsp;&nbsp;&nbsp; A coluna *rssi* também apresenta dados ausentes, embora em menor quantidade. Nesse sentido, o tratamento aplicado a essa coluna foi a substituição dos dados nulos por zero, considerando que a análise se concentra nos valores negativos e nos positivos, sendo que esse último representa uma inconsistência, visto que o correto, como mencionado anteriormente, é apenas valores negativos. Por fim, as linhas que apresentam valores positivos serão mantidas com o objetivo de análises futuras sobre as anômalias

In [99]:
df['rssi'] = df['rssi'].fillna(0)

&nbsp;&nbsp;&nbsp;&nbsp; Além das colunas mencionadas acima, as de 'bairro', 'categoria', 'perfil_consumo' e 'condCode' por conterem poucos valores nulos, o tratamento desses dados foi a substituição dos dados ausentes pela moda da coluna.

In [100]:
for column in ['bairro', 'categoria', 'perfil_consumo', 'condCode']:
    df[column] = df[column].fillna(df[column].mode()[0])


&nbsp;&nbsp;&nbsp;&nbsp; Por fim, após os precessos mencionados, todos os dados nulos foram devidamente trtatados, como é exposto a seguir:

In [ ]:
df.isnull().sum()

## 2.1.3. Codificação das Variáveis Categóricas

&nbsp;&nbsp;&nbsp;&nbsp; O processo de codificação de variáveis categóricas é um conjunto de métodos utilizados para transformar atributos categóricos em representações numéricas. Nesse sentido, essa transformação é essencial para que os algoritmos de *machine learning* possam processar e interpretar os dados corretamente (Passos, 2023). Nesse contexto, a coluna abaixo indica a classificação de cada variável,em seguida, um *DataFrame* é gerado com informações auxiliares para a escolha do melhor método.

In [ ]:
df.info()

In [ ]:
df.select_dtypes(include='object').describe()

 &nbsp;&nbsp;&nbsp;&nbsp; Nas colunas **'clientCode', 'bairro', 'categoria' e 'condCode'** foi aplicado o método *Label Encoding* (codificação de rótulo), na qual para cada variável categórica é atribuido um número inteiro único.

In [104]:
LabelEncoder = LabelEncoder()

df['clientCode'] = LabelEncoder.fit_transform(df['clientCode'])
df['bairro'] = LabelEncoder.fit_transform(df['bairro'])
df['categoria'] = LabelEncoder.fit_transform(df['categoria'])
df['condCode'] = LabelEncoder.fit_transform(df['condCode'])

&nbsp;&nbsp;&nbsp;&nbsp; Na coluna **'inputType'**, também foi aplicado o método *Label Encoding*. No entanto, por conter poucos valores únicos e visando uma melhor identificação, foi mapeado manualmente qual valor seria atribuído a cada variável categórica.

In [ ]:
input_type_mapping = {
    'leituraRemota': 0,
    'DI1': 1,
    'DI2': 2,
    'DI3': 2,
    'DI4': 4,
    'DI5': 5,
    'DI6': 6,
    'DI7': 7,
    'DI8': 8,
}

df['inputType'] = df['inputType'].map(input_type_mapping)

df


&nbsp;&nbsp;&nbsp;&nbsp; O conceito de *Unix Timestamp*, também conhecido como "*Epoch time*", é uma forma de representar a data e hora como um único número, que corresponde ao número de segundos desde 1º de janeiro de 1970, às 00:00:00 UTC (Code BR - Um blog sobre programação, 2024). Dessa forma, esse foi o método utilizado na coluna **'datetime'**.

In [106]:
df['datetime'] = pd.to_datetime(df['datetime']).astype('int64') // 10**9

&nbsp;&nbsp;&nbsp;&nbsp; Nas colunas **'model'** e **'perfil_consumo'**, utilizou-se a codificação *One-Hot*. Essa técnica transforma cada categoria de uma variável em uma nova coluna binária (0 ou 1), sendo que 1 indica presença e 0 indica ausência.

In [ ]:
codificadorModel = OneHotEncoder(cols=['model'])
df = codificadorModel.fit_transform(df)
df.rename(columns={'model_1': 'IG1K-L-v2', 'model_2': 'Infinity V2'}, inplace=True)
df

&nbsp;&nbsp;&nbsp;&nbsp; Antes de aplicar a codificação na coluna **'perfil_consumo'**, foi identificado, ao aplicar o método *unique()* para descobrir os valores únicos, que existe o dado "-" que é inconsistente para o treinamento do modelo preditivo. Considerando que, para a análise do perfil de consumo de cada cliente, essa é uma informação sem valor, as linhas que contêm esses valores foram substituídas pela moda da coluna.

In [ ]:
df['perfil_consumo'].unique()

In [109]:
moda = df['perfil_consumo'].mode()[0]
df = df.replace('-', moda)

In [ ]:
codificadorPerfilConsumo = OneHotEncoder(cols=['perfil_consumo'])
df = codificadorPerfilConsumo.fit_transform(df)
df

&nbsp;&nbsp;&nbsp;&nbsp; Após a codificação por *One-Hot*, a visualização das colunas adicionadas do tipo de consumo tornou-se de difícil entendimento, pois não estava explícito qual variável estava sendo analisada em cada coluna. Tendo isso em vista, foi construído um código que identifica o índice da linha em cada coluna que apresenta valor igual a um, ou seja, de presença. Em seguida, esses índices são acessados no *DataFrame* 'df_auxiliar', cujas informações não estão codificadas, para descobrir o valor correspondente da coluna analisada e, assim, substituí-lo no *DataFrame* original.

In [ ]:
df_auxiliar = pd.merge(meses, cadastro, on=['clientCode', 'clientIndex',], how='left')
df_auxiliar.to_csv('resultado.csv', index=False)

colunasPerfilConsumo = df[['perfil_consumo_1', 'perfil_consumo_2', 'perfil_consumo_3', 
                             'perfil_consumo_4', 'perfil_consumo_5', 'perfil_consumo_6']]

indices = []

for coluna in colunasPerfilConsumo:
    indice = df[df[coluna] == 1].index[0] if not df[df[coluna] == 1].empty else None
    indices.append(indice)

valorIndice = []

for indice in indices:
    valor = df_auxiliar.loc[indice,'perfil_consumo']
    valorIndice.append(valor)

df = df.rename(columns={'perfil_consumo_1': valorIndice[0], 'perfil_consumo_2': valorIndice[1], 'perfil_consumo_3': valorIndice[2], 'perfil_consumo_4': valorIndice[3], 'perfil_consumo_5': valorIndice[4], 'perfil_consumo_6': valorIndice[5] })

df

&nbsp;&nbsp;&nbsp;&nbsp; Por fim, após todo o tratamento das variáveis categóricas, utilizando métodos diversos, todas as colunas passaram a apresentar o tipo numérico.

In [ ]:
df.info()

## 2.2. Normalização das Variáveis Numéricas

&nbsp;&nbsp;&nbsp;&nbsp; Outro tratamento necessário para a realização do pré-processamento é normalização das variáveis numéricas, ou seja, é realizada a transformação dos dados numéricos com o objetivo de que todos eles fiquem com a mesma ordem de grandeza (Junior, 2023). Nesse sentido, nas colunas '*meterIndex*' e '*rssi*' foi aplicado o metódo da normalização *MinMaxScaler*, a qual identifica os valores máximos e mínimo e com isso colocam as variáveis em um intervalo de 0 e 1.

In [ ]:
scaler = MinMaxScaler()
df[['rssi']] = scaler.fit_transform(df[['rssi']])
df

## 3. Escolha das Variáveis Utilizáveis

&nbsp;&nbsp;&nbsp;&nbsp;A seleção das nossas features foi realizada com base em uma análise do que consideramos mais adequado. Como nosso modelo é não supervisionado, não tivemos a oportunidade de testar previamente quais seriam as features mais eficazes. Dessa forma, a escolha foi fundamentada em suposições teóricas, visando capturar o máximo de informação relevante possível. Logo abaixo podemos visualizar quais foram as features selecionadas.

In [ ]:
df.head(0)

## 4. Estatística Descritiva
&nbsp;&nbsp;&nbsp;&nbsp;Estatísticas descritivas são técnicas utilizadas para resumir e descrever as principais características de um conjunto de dados, oferecendo uma visão geral de sua distribuição e variabilidade. Elas facilitam a compreensão dos dados, permitindo que informações complexas sejam transmitidas de forma clara e concisa, sem realizar inferências ou previsões sobre um conjunto maior.
### 4.1. Média

&nbsp;&nbsp;&nbsp;&nbsp;Abaixo, calculamos a média de todas as colunas. A média é o valor obtido ao somar todos os números em uma coluna e dividir pelo total de valores. O gráfico mostra essas médias em uma escala logarítmica para facilitar a visualização, e os valores das médias são exibidos.

In [ ]:
#calcula as médias
medias = df.mean()

cores = sns.color_palette("Set2", len(medias))

#plota o gráfico das médias
plt.figure(figsize=(10, 6))
medias.plot(kind='bar', color=cores)

plt.yscale('log')
plt.title('Média de Cada Coluna')
plt.xlabel('Colunas')
plt.ylabel('Média')
plt.show()

#imprime as médias
medias

### 4.2. Moda

&nbsp;&nbsp;&nbsp;&nbsp;Abaixo, calculamos a moda de todas as colunas. A moda é o valor que aparece com mais frequência em um conjunto de dados. O gráfico mostra essas modas em uma escala logarítmica para melhor visualização, e os valores das modas são exibidos.

In [ ]:
# Calcula as modas
modas = df.mode().iloc[0]  # Seleciona a primeira moda de cada coluna

cores = sns.color_palette("Set2", len(modas))

# Cria o gráfico das modas
plt.figure(figsize=(10, 6))
modas.plot(kind='bar', color=cores)

plt.yscale('log')
plt.title('Moda de Cada Coluna')
plt.xlabel('Colunas')
plt.ylabel('Moda')

# Ajusta o layout para evitar sobreposição
plt.tight_layout()

plt.show()

# Imprime as modas
modas


### 4.3. Mediana

&nbsp;&nbsp;&nbsp;&nbsp;Abaixo, calculamos a mediana de todas as colunas. A mediana é o valor que está no meio de um conjunto de dados quando eles estão ordenados. Se houver um número par de valores, a mediana é a média dos dois valores centrais. O gráfico exibe essas medianas em uma escala logarítmica para facilitar a visualização, e os valores das medianas são apresentados.

In [ ]:
# Calcula a mediana de cada coluna
medianas = df.median()

cores = sns.color_palette("Set2", len(medianas))

# Cria o gráfico das medianas
plt.figure(figsize=(10, 6))
medianas.plot(kind='bar', color=cores)

plt.yscale('log')
plt.title('Mediana de Cada Coluna')
plt.xlabel('Colunas')
plt.ylabel('Mediana')

# Ajusta automaticamente o layout para evitar sobreposição
plt.tight_layout()

plt.show()

# Imprime as medianas
medianas

### 4.4. Desvio Padrão

&nbsp;&nbsp;&nbsp;&nbsp;Abaixo, calculamos o desvio padrão de todas as colunas. O desvio padrão é a raiz quadrada da variância e mede o quanto os valores se afastam da média. O gráfico mostra esses desvios padrão em uma escala logarítmica para facilitar a visualização, e os valores dos desvios padrão são apresentados.

In [ ]:
# Calcula o desvio padrão de cada coluna
desvios_padrao = df.std()

cores = sns.color_palette("Set2", len(desvios_padrao))

# Cria o gráfico dos desvios padrão
plt.figure(figsize=(10, 6))
desvios_padrao.plot(kind='bar', color=cores)

plt.yscale('log')
plt.title('Desvio Padrão de Cada Coluna')
plt.xlabel('Colunas')
plt.ylabel('Desvio Padrão')

# Ajusta automaticamente o layout para evitar sobreposição
plt.tight_layout()

plt.show()

# Imprime os desvios padrão
desvios_padrao

### 4.5. Variância

&nbsp;&nbsp;&nbsp;&nbsp;Abaixo, calculamos a variância de todas as colunas. A variância mede a dispersão dos valores em torno da média, indicando o quanto eles variam. O gráfico mostra essas variâncias em uma escala logarítmica para melhorar a visualização, e os valores das variâncias são exibidos.

In [ ]:
# Calcula a variância de cada coluna
variancias = df.var()

cores = sns.color_palette("Set2", len(variancias))

# Cria o gráfico das variâncias
plt.figure(figsize=(10, 6))
variancias.plot(kind='bar', color=cores)

plt.yscale('log')
plt.title('Variância de Cada Coluna')
plt.xlabel('Colunas')
plt.ylabel('Variância')

# Ajusta automaticamente o layout para evitar sobreposição
plt.tight_layout()

plt.show()

# Imprime as variâncias
variancias

## 5. Hipóteses

&nbsp;&nbsp;&nbsp;&nbsp;Levantar hipóteses durante o progresso do modelo preditivo é extremamente importante para termos uma tese que nos guiará durante tomadas de decisão, divisão de tarefas e diversas outras atividade que acompanham o desenvolvimento.

### 5.1. Hipótese 1

&nbsp;&nbsp;&nbsp;&nbsp;A primeira hipótese levantada pelo grupo é que os usuários individuais de gás encanado têm um padrão de consumo muito semelhante entre si. Verificamos essa tese quando geramos o gráfico que mostra o consumo dos usuários durante os meses.

In [ ]:
#Gráfico que gera o consumo do gás durante os 5 meses
sns.boxplot(df['meterIndex'])

&nbsp;&nbsp;&nbsp;&nbsp;Com esse gráfico, conseguimos perceber que os usuários estão agrupados no meio do gráfico, mostrando um consumo médio perto dos 30000 metros cúbicos de gás, muito diferente dos *outliers* identificados, que passam de 300000 metros cúbicos consumidos.

### 5.2. Hipótese 2

&nbsp;&nbsp;&nbsp;&nbsp;A segunda hipótese é de que o mês de março tem a maior média de consumo comparado aos demais meses. Acreditamos que esse fato ocorre principalmente pela transição entre o verão e outono, o que causa temperaturas inconstantes durante esse mês. Podemos ver os padrões de consumo com o código abaixo.

In [ ]:
#Código que gera a média de consumo do mês de fevereiro de 2024
mes_02['meterIndex'].mean

In [ ]:
#Código que gera a média de consumo do mês de março de 2024
mes_03['meterIndex'].mean

In [ ]:
#Código que gera a média de consumo do mês de abril de 2024
mes_04['meterIndex'].mean

In [ ]:
#Código que gera a média de consumo do mês de maio de 2024
mes_05['meterIndex'].mean

In [ ]:
#Código que gera a média de consumo do mês de junho de 2024
mes_06['meterIndex'].mean

### 5.3. Hipótese 3

&nbsp;&nbsp;&nbsp;&nbsp;Chegamos à terceira hipótese analisando o mês de junho de 2024, último mês do nosso banco de dados. Notamos que ao contrário do mês de março, a média de consumo desse mês é mais baixa. Com isso, notamos que o mês de junho de 2024 foi data de um ciclone no estado do Rio Grande do Sul, o que pode ter acarretado em um atraso na rede de distribuição do gás e consequentemente, um consumo menor por parte dos usuários. Podemos ver isso com o código abaixo.

In [ ]:
#Código que gera a média do mês de junho de 2024
mes_06['meterIndex'].mean

&nbsp;&nbsp;&nbsp;&nbsp;Adicionando mais informações ao levantamento dessa hipóteses, vimos que o referido ciclone extratropical não foi tão forte, mas pode ter prejudicado parte da distruição do gás encanado.

### 6. Adição de colunas extras conforme a demanda dos modelos

&nbsp;&nbsp;&nbsp;&nbsp;Adiciona coluna de gastos tendo em vista o feedback do cliente, pois, o meterIndex é um fator cumulativo, ou seja, sempre aumenta.

In [ ]:
df['gain'] = df['gain'].fillna(1)
df['medidor'] = df['pulseCount'] * df['gain']
df['gasto'] = 0.0

df = df.sort_values(by=['datetime', 'clientCode', ])

df['gasto'] = df.groupby(['clientCode', 'clientIndex'])['medidor'].diff()

df['gasto'] = df['gasto'].fillna(0)

resultado = df

df['gasto'] = df['gasto'].fillna(0)


df = df.sort_values(by=[ 'clientCode', 'clientIndex','datetime'])
resultado = df

# verifica se há valores negativos
print("\nLinhas com 'gasto' negativo:")
print(resultado[resultado['gasto'] < 0])

Verifica se há números negativos na coluna gastos,
já que a coluna gastos pelo número seguinte sempre ser maior que o anterior não deve ter números negativos:

In [ ]:
gastos_negativos = df[df['gasto'] < 0]
print(gastos_negativos)

In [129]:
df.to_csv('../data/dataframe.csv', index=False)